## **Toxic comments**

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
%cd /content/gdrive/My Drive/HSE NLP

/content/gdrive/My Drive/HSE NLP


In [ ]:
!pip install pymorphy2

     |████████████████████████████████| 55 kB 1.8 MB/s 
     |████████████████████████████████| 8.2 MB 12.1 MB/s 


In [ ]:
!pip install transliterate

     |████████████████████████████████| 45 kB 1.7 MB/s  eta 0:00:01


In [ ]:
!pip install wordcloud

In [ ]:
import functools
import pymorphy2
import re
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from multiprocessing import Pool
from transliterate import translit
from wordcloud import WordCloud
from nltk import ngrams
from nltk.corpus import stopwords
from nltk.probability import FreqDist
from tqdm.notebook import tqdm
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import *
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression 
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer

pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', 800)

%matplotlib inline

In [ ]:
# после предобработки текстов (лемматизации, разбивки на предложения и т.д.) я сохраняла эти датафреймы, чтобы в дальнейшей работе не проделывать это всё заново:
df_train = pd.read_csv('df_train.csv')
df_test = pd.read_csv('df_test.csv')

## **Train/ test split**

In [ ]:
scaler = StandardScaler()
df_train['len_norm'] = np.round(scaler.fit_transform(df_train[['comment_len']]),3)
df_train['len_log_norm'] = np.round(scaler.fit_transform(df_train[['comment_len_log']]),3)

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(df_train[['lemmas', 'sent_lemmas', 'len_norm', 'len_log_norm']], df_train['toxic'])

In [ ]:
x_train.shape, x_val.shape

((8106, 4), (2703, 4))

In [ ]:
grid_searcher = GridSearchCV(
    LogisticRegression(max_iter=3000, solver='liblinear', random_state=42),
    param_grid={
        'C': np.logspace(-5, 1, 6),
        'penalty': ['l1', 'l2']
    },
    cv=10,
    scoring='accuracy',
    n_jobs=-1,
    verbose=5)

## **fastText => вектора для текстов**

In [ ]:
#!wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.ru.300.bin.gz

--2021-10-25 17:19:06--  https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.ru.300.bin.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 172.67.9.4, 104.22.75.142, 104.22.74.142, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|172.67.9.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4496459151 (4.2G) [application/octet-stream]
Saving to: ‘cc.ru.300.bin.gz’

cc.ru.300.bin.gz    100%[===================>]   4.19G  23.8MB/s    in 3m 9s   

2021-10-25 17:22:16 (22.7 MB/s) - ‘cc.ru.300.bin.gz’ saved [4496459151/4496459151]



In [ ]:
!git clone https://github.com/facebookresearch/fastText.git
%cd /content/gdrive/My Drive/HSE NLP/fastText
!sudo pip install .

fatal: destination path 'fastText' already exists and is not an empty directory.
/content/gdrive/My Drive/HSE NLP/fastText
Processing /content/gdrive/My Drive/HSE NLP/fastText
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
  Using cached pybind11-2.8.1-py2.py3-none-any.whl (208 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp37-cp37m-linux_x86_64.whl size=3120429 sha256=8a343d048b5d95af4264c31d7ff442fddb37778ac4b5f96390569bdc274db547
  Stored in directory: /tmp/pip-ephem-wheel-cache-vcebe9fy/wheels/2b/b2/ff/8a0b9862025fb0b9f619755659e0189e7b2888ea2fa68799bc
Successfully built fasttext


In [ ]:
#import fasttext.util
#fasttext.util.download_model('ru', if_exists='ignore')
#ft = fasttext.load_model('cc.ru.300.bin')
#ft.save_model("fasttext_imported_model.bin")

In [ ]:
import fasttext.util
ft = fasttext.load_model("/content/gdrive/My Drive/HSE NLP/fasttext_imported_model.bin")

**Предобученная модель - Без разбивки на предложения**

In [ ]:
vec_x_train = x_train['lemmas'].apply(ft.get_sentence_vector)
vec_x_val = x_val['lemmas'].apply(ft.get_sentence_vector)

In [ ]:
vec_x_train = {'vectors': vec_x_train}
vec_x_train = pd.DataFrame(vec_x_train)
vec_x_train = pd.DataFrame(vec_x_train.vectors.tolist(), index= vec_x_train.index)

In [ ]:
vec_x_val = {'vectors': vec_x_val}
vec_x_val = pd.DataFrame(vec_x_val)
vec_x_val = pd.DataFrame(vec_x_val.vectors.tolist(), index= vec_x_val.index)

In [ ]:
vec_x_train.shape, vec_x_val.shape

((8106, 300), (2703, 300))

In [ ]:
grid_searcher.fit(vec_x_train, y_train)

Fitting 10 folds for each of 12 candidates, totalling 120 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:    9.0s
[Parallel(n_jobs=-1)]: Done  68 tasks      | elapsed:   15.6s
[Parallel(n_jobs=-1)]: Done 120 out of 120 | elapsed:  1.1min finished


GridSearchCV(cv=10, error_score=nan,
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=3000, multi_class='auto',
                                          n_jobs=None, penalty='l2',
                                          random_state=42, solver='liblinear',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='deprecated', n_jobs=-1,
             param_grid={'C': array([1.00000000e-05, 1.58489319e-04, 2.51188643e-03, 3.98107171e-02,
       6.30957344e-01, 1.00000000e+01]),
                         'penalty': ['l1', 'l2']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='accuracy', verbose=5)

In [ ]:
pd.DataFrame(grid_searcher.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_penalty,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
0,0.173302,0.056521,0.008855,0.009854,1e-05,l1,"{'C': 1e-05, 'penalty': 'l1'}",0.664612,0.664612,0.664612,0.664612,0.664612,0.664612,0.665432,0.664198,0.664198,0.664198,0.664569,0.000342,7
1,0.171192,0.046944,0.003141,0.000133,1e-05,l2,"{'C': 1e-05, 'penalty': 'l2'}",0.664612,0.664612,0.664612,0.664612,0.664612,0.664612,0.665432,0.664198,0.664198,0.664198,0.664569,0.000342,7
2,0.148895,0.029475,0.006234,0.007049,0.000158489,l1,"{'C': 0.00015848931924611142, 'penalty': 'l1'}",0.664612,0.664612,0.664612,0.664612,0.664612,0.664612,0.665432,0.664198,0.664198,0.664198,0.664569,0.000342,7
3,0.249934,0.060423,0.006684,0.006650,0.000158489,l2,"{'C': 0.00015848931924611142, 'penalty': 'l2'}",0.664612,0.664612,0.664612,0.664612,0.664612,0.664612,0.665432,0.664198,0.664198,0.664198,0.664569,0.000342,7
4,0.173596,0.042142,0.003147,0.000197,0.00251189,l1,"{'C': 0.0025118864315095794, 'penalty': 'l1'}",0.664612,0.664612,0.664612,0.664612,0.664612,0.664612,0.665432,0.664198,0.664198,0.664198,0.664569,0.000342,7
5,0.253953,0.038804,0.004627,0.004275,0.00251189,l2,"{'C': 0.0025118864315095794, 'penalty': 'l2'}",0.664612,0.664612,0.664612,0.664612,0.664612,0.664612,0.665432,0.664198,0.664198,0.664198,0.664569,0.000342,7
6,0.325208,0.038287,0.006884,0.006086,0.0398107,l1,"{'C': 0.03981071705534969, 'penalty': 'l1'}",0.803946,0.807645,0.824908,0.803946,0.794081,0.811344,0.816049,0.841975,0.834568,0.830864,0.816933,0.014743,5
7,0.283951,0.031485,0.004072,0.002663,0.0398107,l2,"{'C': 0.03981071705534969, 'penalty': 'l2'}",0.759556,0.778052,0.764488,0.755857,0.766954,0.760789,0.762963,0.790123,0.790123,0.774074,0.770298,0.011720,6
8,2.173646,0.313817,0.008065,0.008246,0.630957,l1,"{'C': 0.630957344480193, 'penalty': 'l1'}",0.864365,0.885327,0.898890,0.869297,0.875462,0.886560,0.898765,0.890123,0.896296,0.892593,0.885768,0.011636,3
9,0.495724,0.062454,0.004844,0.004532,0.630957,l2,"{'C': 0.630957344480193, 'penalty': 'l2'}",0.864365,0.880395,0.881628,0.863132,0.865598,0.876695,0.886420,0.883951,0.895062,0.887654,0.878490,0.010342,4


In [ ]:
grid_searcher.best_params_

{'C': 10.0, 'penalty': 'l2'}

In [ ]:
lr = grid_searcher.best_estimator_

In [ ]:
lr.fit(vec_x_train, y_train)
pred = lr.predict(vec_x_val)
print(classification_report(pred, y_val))

              precision    recall  f1-score   support

           0       0.94      0.91      0.92      1877
           1       0.81      0.87      0.84       826

    accuracy                           0.90      2703
   macro avg       0.87      0.89      0.88      2703
weighted avg       0.90      0.90      0.90      2703



**Предобученная модель - Без разбики на предложения - Для submission**

In [ ]:
vec_train = df_train['lemmas'].apply(ft.get_sentence_vector)

In [ ]:
vec_test = df_test['lemmas'].apply(ft.get_sentence_vector)

In [ ]:
vec_df_train = {'vectors': vec_train}
vec_df_train = pd.DataFrame(vec_df_train)
vec_df_train = pd.DataFrame(vec_df_train.vectors.tolist(), index= vec_df_train.index)

In [ ]:
vec_df_test = {'vectors': vec_test}
vec_df_test = pd.DataFrame(vec_df_test)
vec_df_test = pd.DataFrame(vec_df_test.vectors.tolist(), index= vec_df_test.index)

In [ ]:
vec_df_train.shape, vec_df_test.shape

((10809, 300), (3603, 300))

In [ ]:
grid_searcher.fit(vec_df_train, df_train['toxic'])

Fitting 10 folds for each of 12 candidates, totalling 120 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:    7.9s
[Parallel(n_jobs=-1)]: Done  68 tasks      | elapsed:   17.1s
[Parallel(n_jobs=-1)]: Done 120 out of 120 | elapsed:  1.2min finished


GridSearchCV(cv=10, error_score=nan,
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=3000, multi_class='auto',
                                          n_jobs=None, penalty='l2',
                                          random_state=42, solver='liblinear',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='deprecated', n_jobs=-1,
             param_grid={'C': array([1.00000000e-05, 1.58489319e-04, 2.51188643e-03, 3.98107171e-02,
       6.30957344e-01, 1.00000000e+01]),
                         'penalty': ['l1', 'l2']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='accuracy', verbose=5)

In [ ]:
pd.DataFrame(grid_searcher.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_penalty,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
0,0.257489,0.074539,0.005807,0.005635,1e-05,l1,"{'C': 1e-05, 'penalty': 'l1'}",0.666975,0.666050,0.666050,0.666050,0.666050,0.666050,0.666050,0.666050,0.666050,0.666667,0.666204,0.000316,7
1,0.246196,0.036795,0.004774,0.001858,1e-05,l2,"{'C': 1e-05, 'penalty': 'l2'}",0.666975,0.666050,0.666050,0.666050,0.666050,0.666050,0.666050,0.666050,0.666050,0.666667,0.666204,0.000316,7
2,0.214145,0.016206,0.006171,0.006643,0.000158489,l1,"{'C': 0.00015848931924611142, 'penalty': 'l1'}",0.666975,0.666050,0.666050,0.666050,0.666050,0.666050,0.666050,0.666050,0.666050,0.666667,0.666204,0.000316,7
3,0.364495,0.021099,0.005765,0.003378,0.000158489,l2,"{'C': 0.00015848931924611142, 'penalty': 'l2'}",0.666975,0.666050,0.666050,0.666050,0.666050,0.666050,0.666050,0.666050,0.666050,0.666667,0.666204,0.000316,7
4,0.259585,0.014530,0.006657,0.005484,0.00251189,l1,"{'C': 0.0025118864315095794, 'penalty': 'l1'}",0.666975,0.666050,0.666050,0.666050,0.666050,0.666050,0.666050,0.666050,0.666050,0.666667,0.666204,0.000316,7
5,0.366127,0.016078,0.008819,0.007953,0.00251189,l2,"{'C': 0.0025118864315095794, 'penalty': 'l2'}",0.666975,0.666050,0.666050,0.666050,0.666050,0.666050,0.666050,0.666050,0.666050,0.666667,0.666204,0.000316,7
6,0.439167,0.066250,0.004289,0.001529,0.0398107,l1,"{'C': 0.03981071705534969, 'penalty': 'l1'}",0.834413,0.827937,0.818686,0.827012,0.817761,0.838113,0.837188,0.821462,0.834413,0.828704,0.828569,0.007082,5
7,0.522535,0.070054,0.003870,0.001132,0.0398107,l2,"{'C': 0.03981071705534969, 'penalty': 'l2'}",0.812211,0.800185,0.785384,0.793710,0.800185,0.792784,0.811286,0.792784,0.795560,0.788889,0.797298,0.008391,6
8,3.088778,0.297885,0.003433,0.000115,0.630957,l1,"{'C': 0.630957344480193, 'penalty': 'l1'}",0.883441,0.891767,0.874191,0.889917,0.887142,0.891767,0.892692,0.889917,0.893617,0.892593,0.888704,0.005636,3
9,0.602570,0.057495,0.003401,0.000064,0.630957,l2,"{'C': 0.630957344480193, 'penalty': 'l2'}",0.876966,0.887142,0.868640,0.882516,0.869565,0.880666,0.892692,0.883441,0.888992,0.887037,0.881766,0.007596,4


In [ ]:
grid_searcher.best_params_

{'C': 10.0, 'penalty': 'l2'}

In [ ]:
lr = grid_searcher.best_estimator_

In [ ]:
lr.fit(vec_df_train, df_train['toxic'])
pred = lr.predict(vec_df_test)

In [ ]:
submission4 = df_test[['comment_id']]
submission4['toxic'] = pred
submission4.to_csv('submission4.csv', index=False)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
df_test['toxic_fasttext'] = pred

In [ ]:
df_train.to_csv('df_train.csv', index=False)
df_test.to_csv('df_test.csv', index=False)

Эта модель после submission получила accuracy 0.89286.

**Предобученная модель - С разбикой на преждложения**

! Лучше сделать разбивку на предложения заново, иначе при импорте листы превращаются в строки

Формула взята отсюда: https://stackoverflow.com/questions/54181163/fasttext-embeddings-sentence-vectors, только без EOS - тогда получается то же, что и в fasttext.get_sentence_vector

In [ ]:
import numpy as np

In [ ]:
def l2_norm(x):
   return np.sqrt(np.sum(x**2))

def div_norm(x):
   norm_value = l2_norm(x)
   if norm_value > 0:
       return x * ( 1.0 / norm_value)
   else:
       return x

In [ ]:
# средний вектор для текста в зависимости от векторов входящих в него предложений
def fasttext_sentences(sentences_list):
  return np.mean(np.array([div_norm(x) for x in np.array(list(map(ft.get_sentence_vector, [' '.join(sl) for sl in sentences_list])))]), 0)

In [ ]:
vec_x_train = x_train['sent_lemmas'].apply(fasttext_sentences)
vec_x_val = x_val['sent_lemmas'].apply(fasttext_sentences)

In [ ]:
vec_x_train = {'vectors': vec_x_train}
vec_x_train = pd.DataFrame(vec_x_train)
vec_x_train = pd.DataFrame(vec_x_train.vectors.tolist(), index= vec_x_train.index)

In [ ]:
vec_x_val = {'vectors': vec_x_val}
vec_x_val = pd.DataFrame(vec_x_val)
vec_x_val = pd.DataFrame(vec_x_val.vectors.tolist(), index= vec_x_val.index)

In [ ]:
vec_x_train.shape, vec_x_val.shape

((8106, 300), (2703, 300))

In [ ]:
grid_searcher.fit(vec_x_train, y_train)

Fitting 10 folds for each of 12 candidates, totalling 120 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:    2.9s
[Parallel(n_jobs=-1)]: Done  68 tasks      | elapsed:    7.3s
[Parallel(n_jobs=-1)]: Done 120 out of 120 | elapsed:   30.7s finished


GridSearchCV(cv=10, error_score=nan,
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=3000, multi_class='auto',
                                          n_jobs=None, penalty='l2',
                                          random_state=42, solver='liblinear',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='deprecated', n_jobs=-1,
             param_grid={'C': array([1.00000000e-05, 1.58489319e-04, 2.51188643e-03, 3.98107171e-02,
       6.30957344e-01, 1.00000000e+01]),
                         'penalty': ['l1', 'l2']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='accuracy', verbose=5)

In [ ]:
pd.DataFrame(grid_searcher.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_penalty,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
0,0.134992,0.044210,0.003160,0.000637,1e-05,l1,"{'C': 1e-05, 'penalty': 'l1'}",0.667078,0.667078,0.665845,0.665845,0.665845,0.665845,0.666667,0.666667,0.666667,0.666667,0.666420,0.000493,7
1,0.136223,0.002959,0.003330,0.001699,1e-05,l2,"{'C': 1e-05, 'penalty': 'l2'}",0.667078,0.667078,0.665845,0.665845,0.665845,0.665845,0.666667,0.666667,0.666667,0.666667,0.666420,0.000493,7
2,0.105634,0.007421,0.003228,0.001427,0.000158489,l1,"{'C': 0.00015848931924611142, 'penalty': 'l1'}",0.667078,0.667078,0.665845,0.665845,0.665845,0.665845,0.666667,0.666667,0.666667,0.666667,0.666420,0.000493,7
3,0.184997,0.005208,0.002792,0.000327,0.000158489,l2,"{'C': 0.00015848931924611142, 'penalty': 'l2'}",0.667078,0.667078,0.665845,0.665845,0.665845,0.665845,0.666667,0.666667,0.666667,0.666667,0.666420,0.000493,7
4,0.124706,0.006534,0.002974,0.000620,0.00251189,l1,"{'C': 0.0025118864315095794, 'penalty': 'l1'}",0.667078,0.667078,0.665845,0.665845,0.665845,0.665845,0.666667,0.666667,0.666667,0.666667,0.666420,0.000493,7
5,0.167961,0.003386,0.002825,0.000116,0.00251189,l2,"{'C': 0.0025118864315095794, 'penalty': 'l2'}",0.667078,0.667078,0.665845,0.665845,0.665845,0.665845,0.666667,0.666667,0.666667,0.666667,0.666420,0.000493,7
6,0.206122,0.012184,0.002999,0.000722,0.0398107,l1,"{'C': 0.03981071705534969, 'penalty': 'l1'}",0.827374,0.828607,0.834772,0.828607,0.832306,0.833539,0.838272,0.835802,0.834568,0.840741,0.833459,0.004124,6
7,0.232488,0.006951,0.002725,0.000106,0.0398107,l2,"{'C': 0.03981071705534969, 'penalty': 'l2'}",0.832306,0.827374,0.842170,0.837238,0.845869,0.847102,0.848148,0.827160,0.843210,0.834568,0.838515,0.007516,5
8,0.666216,0.068866,0.002766,0.000092,0.630957,l1,"{'C': 0.630957344480193, 'penalty': 'l1'}",0.876695,0.882861,0.881628,0.890259,0.890259,0.887793,0.880247,0.897531,0.886420,0.896296,0.886999,0.006460,3
9,0.404755,0.010882,0.002817,0.000081,0.630957,l2,"{'C': 0.630957344480193, 'penalty': 'l2'}",0.866831,0.875462,0.866831,0.889026,0.889026,0.882861,0.880247,0.886420,0.875309,0.898765,0.881078,0.009708,4


In [ ]:
grid_searcher.best_params_

{'C': 10.0, 'penalty': 'l2'}

In [ ]:
lr = grid_searcher.best_estimator_

In [ ]:
lr.fit(vec_x_train, y_train)
pred = lr.predict(vec_x_val)
print(classification_report(pred, y_val))

              precision    recall  f1-score   support

           0       0.95      0.90      0.92      1900
           1       0.79      0.89      0.83       803

    accuracy                           0.90      2703
   macro avg       0.87      0.89      0.88      2703
weighted avg       0.90      0.90      0.90      2703



**Предобученная модель - С разбивкой на предложения - Для submission**

In [ ]:
vec_train = df_train['sent_lemmas'].apply(fasttext_sentences)

In [ ]:
vec_test = df_test['sent_lemmas'].apply(fasttext_sentences)

In [ ]:
vec_df_train = {'vectors': vec_train}
vec_df_train = pd.DataFrame(vec_df_train)
vec_df_train = pd.DataFrame(vec_df_train.vectors.tolist(), index= vec_df_train.index)

In [ ]:
vec_df_test = {'vectors': vec_test}
vec_df_test = pd.DataFrame(vec_df_test)
vec_df_test = pd.DataFrame(vec_df_test.vectors.tolist(), index= vec_df_test.index)

In [ ]:
vec_df_train.shape, vec_df_test.shape

((10809, 300), (3603, 300))

In [ ]:
grid_searcher.fit(vec_df_train, df_train['toxic'])

Fitting 10 folds for each of 12 candidates, totalling 120 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:    1.3s
[Parallel(n_jobs=-1)]: Done  68 tasks      | elapsed:    7.4s
[Parallel(n_jobs=-1)]: Done 120 out of 120 | elapsed:   40.3s finished


GridSearchCV(cv=10, error_score=nan,
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=3000, multi_class='auto',
                                          n_jobs=None, penalty='l2',
                                          random_state=42, solver='liblinear',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='deprecated', n_jobs=-1,
             param_grid={'C': array([1.00000000e-05, 1.58489319e-04, 2.51188643e-03, 3.98107171e-02,
       6.30957344e-01, 1.00000000e+01]),
                         'penalty': ['l1', 'l2']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='accuracy', verbose=5)

In [ ]:
pd.DataFrame(grid_searcher.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_penalty,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
0,0.154862,0.012780,0.003405,0.001277,1e-05,l1,"{'C': 1e-05, 'penalty': 'l1'}",0.666975,0.666050,0.666050,0.666050,0.666050,0.666050,0.666050,0.666050,0.666050,0.666667,0.666204,0.000316,8
1,0.198886,0.006708,0.003451,0.001650,1e-05,l2,"{'C': 1e-05, 'penalty': 'l2'}",0.666975,0.666050,0.666050,0.666050,0.666050,0.666050,0.666050,0.666050,0.666050,0.666667,0.666204,0.000316,8
2,0.152718,0.007560,0.003033,0.000092,0.000158489,l1,"{'C': 0.00015848931924611142, 'penalty': 'l1'}",0.666975,0.666050,0.666050,0.666050,0.666050,0.666050,0.666050,0.666050,0.666050,0.666667,0.666204,0.000316,8
3,0.211352,0.009270,0.003064,0.000097,0.000158489,l2,"{'C': 0.00015848931924611142, 'penalty': 'l2'}",0.666975,0.666050,0.666050,0.666050,0.666050,0.666050,0.666050,0.666050,0.666050,0.666667,0.666204,0.000316,8
4,0.178524,0.012147,0.002878,0.000183,0.00251189,l1,"{'C': 0.0025118864315095794, 'penalty': 'l1'}",0.666975,0.666050,0.666050,0.666050,0.666050,0.666050,0.666050,0.666050,0.666050,0.666667,0.666204,0.000316,8
5,0.244788,0.006169,0.003141,0.000466,0.00251189,l2,"{'C': 0.0025118864315095794, 'penalty': 'l2'}",0.666975,0.667900,0.666050,0.666975,0.670675,0.666050,0.666050,0.666975,0.667900,0.667593,0.667314,0.001312,7
6,0.320169,0.030638,0.003155,0.000302,0.0398107,l1,"{'C': 0.03981071705534969, 'penalty': 'l1'}",0.849214,0.849214,0.834413,0.828862,0.825162,0.847364,0.851989,0.842738,0.841813,0.839815,0.841058,0.008619,6
7,0.336490,0.008712,0.003142,0.000217,0.0398107,l2,"{'C': 0.03981071705534969, 'penalty': 'l2'}",0.864015,0.858464,0.839038,0.844588,0.847364,0.848289,0.863090,0.858464,0.845513,0.846296,0.851512,0.008254,5
8,1.043987,0.036470,0.003031,0.000463,0.630957,l1,"{'C': 0.630957344480193, 'penalty': 'l1'}",0.891767,0.894542,0.881591,0.881591,0.892692,0.894542,0.900093,0.894542,0.894542,0.891667,0.891757,0.005550,3
9,0.609813,0.010622,0.003127,0.000088,0.630957,l2,"{'C': 0.630957344480193, 'penalty': 'l2'}",0.885291,0.892692,0.872340,0.876966,0.884366,0.880666,0.896392,0.894542,0.892692,0.887037,0.886299,0.007553,4


In [ ]:
grid_searcher.best_params_

{'C': 10.0, 'penalty': 'l2'}

In [ ]:
lr = grid_searcher.best_estimator_

In [ ]:
lr.fit(vec_df_train, df_train['toxic'])
pred = lr.predict(vec_df_test)

In [ ]:
submission5 = df_test[['comment_id']]
submission5['toxic'] = pred
submission5.to_csv('submission5.csv', index=False)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


В результате получился тот же скор, что и без разбивки на предложения - 0.89286.

## **Дообучение fastText**

! При загрузке датафреймов из csv лучше сделать разбивку на предложения заново, иначе при импорте листы превращаются в строки

Непонятно, можно ли дообучить модель из fastText

In [ ]:
sentences_list_train = [s for sl in df_train['sent_lemmas'] for s in sl]
sentences_list_test = [s for sl in df_test['sent_lemmas'] for s in sl]

In [ ]:
sentences_list_x_train = [s for sl in x_train['sent_lemmas'] for s in sl]
sentences_list_x_val = [s for sl in x_val['sent_lemmas'] for s in sl]

## **t-SNE (после предобученной модели fastText (ft))**

In [ ]:
freq_words = fdist_all['lemma'][:500]

In [ ]:
freq_words = freq_words.to_list()
freq_words_vec = [ft[w] for w in freq_words]

In [ ]:
from sklearn.manifold import TSNE

In [ ]:
tsne = TSNE(n_components=2, random_state=42)
freq_words_tsne = tsne.fit_transform(freq_words_vec)

In [ ]:
from bokeh.models import ColumnDataSource, LabelSet
from bokeh.plotting import figure, show, output_file
from bokeh.io import output_notebook
output_notebook()

p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="t-SNE (eng model, top 500 words)")

source = ColumnDataSource(data=dict(x1=freq_words_tsne[:,0],
                                    x2=freq_words_tsne[:,1],
                                    names=freq_words))

p.scatter(x="x1", y="x2", size=8, source=source)

labels = LabelSet(x="x1", y="x2", text="names", y_offset=6,
                  text_font_size="8pt", text_color="#555555",
                  source=source, text_align='center')
p.add_layout(labels)

show(p)

## **Обучение fastText, unsupervised (без разбивки по предложениям)**

С разбивкой по предложениям тоже учила - качество было хуже, поэтому я удалила эту часть кода.

In [ ]:
with open('lemmas_train.txt', 'w') as f:
    for s in df_train['lemmas'].tolist():
        f.write(s)
        f.write('\n')

In [ ]:
import fasttext.util
ft_train = fasttext.train_unsupervised('lemmas_train.txt', minn=2, maxn=5, dim=300, epoch=50)

In [ ]:
vec_x_train = x_train['lemmas'].apply(ft_train.get_sentence_vector)
vec_x_val = x_val['lemmas'].apply(ft_train.get_sentence_vector)

In [ ]:
vec_x_train = {'vectors': vec_x_train}
vec_x_train = pd.DataFrame(vec_x_train)
vec_x_train = pd.DataFrame(vec_x_train.vectors.tolist(), index= vec_x_train.index)

In [ ]:
vec_x_val = {'vectors': vec_x_val}
vec_x_val = pd.DataFrame(vec_x_val)
vec_x_val = pd.DataFrame(vec_x_val.vectors.tolist(), index= vec_x_val.index)

In [ ]:
vec_x_train.shape, vec_x_val.shape

((8100, 300), (2701, 300))

In [ ]:
grid_searcher.fit(vec_x_train, y_train)

Fitting 10 folds for each of 12 candidates, totalling 120 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:    3.1s
[Parallel(n_jobs=-1)]: Done  68 tasks      | elapsed:    8.8s
[Parallel(n_jobs=-1)]: Done 120 out of 120 | elapsed:  1.7min finished


GridSearchCV(cv=10, error_score=nan,
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=3000, multi_class='auto',
                                          n_jobs=None, penalty='l2',
                                          random_state=42, solver='liblinear',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='deprecated', n_jobs=-1,
             param_grid={'C': array([1.00000000e-05, 1.58489319e-04, 2.51188643e-03, 3.98107171e-02,
       6.30957344e-01, 1.00000000e+01]),
                         'penalty': ['l1', 'l2']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='accuracy', verbose=5)

In [ ]:
pd.DataFrame(grid_searcher.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_penalty,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
0,0.157940,0.035186,0.003783,0.000871,1e-05,l1,"{'C': 1e-05, 'penalty': 'l1'}",0.665432,0.665432,0.665432,0.665432,0.665432,0.665432,0.665432,0.665432,0.665432,0.664198,0.665309,0.000370,7
1,0.155965,0.016062,0.004410,0.001268,1e-05,l2,"{'C': 1e-05, 'penalty': 'l2'}",0.665432,0.665432,0.665432,0.665432,0.665432,0.665432,0.665432,0.665432,0.665432,0.664198,0.665309,0.000370,7
2,0.137013,0.008240,0.003565,0.000438,0.000158489,l1,"{'C': 0.00015848931924611142, 'penalty': 'l1'}",0.665432,0.665432,0.665432,0.665432,0.665432,0.665432,0.665432,0.665432,0.665432,0.664198,0.665309,0.000370,7
3,0.231686,0.008761,0.004243,0.001534,0.000158489,l2,"{'C': 0.00015848931924611142, 'penalty': 'l2'}",0.665432,0.665432,0.665432,0.665432,0.665432,0.665432,0.665432,0.665432,0.665432,0.664198,0.665309,0.000370,7
4,0.160724,0.004870,0.003529,0.000436,0.00251189,l1,"{'C': 0.0025118864315095794, 'penalty': 'l1'}",0.665432,0.665432,0.665432,0.665432,0.665432,0.665432,0.665432,0.665432,0.665432,0.664198,0.665309,0.000370,7
5,0.218236,0.010469,0.004291,0.001715,0.00251189,l2,"{'C': 0.0025118864315095794, 'penalty': 'l2'}",0.665432,0.665432,0.665432,0.665432,0.665432,0.665432,0.665432,0.665432,0.665432,0.664198,0.665309,0.000370,7
6,0.273726,0.019777,0.003426,0.000252,0.0398107,l1,"{'C': 0.03981071705534969, 'penalty': 'l1'}",0.665432,0.665432,0.666667,0.666667,0.665432,0.665432,0.665432,0.665432,0.665432,0.664198,0.665556,0.000665,6
7,0.269502,0.033857,0.003275,0.000188,0.0398107,l2,"{'C': 0.03981071705534969, 'penalty': 'l2'}",0.714815,0.695062,0.707407,0.717284,0.704938,0.717284,0.708642,0.718519,0.700000,0.709877,0.709383,0.007453,5
8,3.057562,0.131996,0.003918,0.000571,0.630957,l1,"{'C': 0.630957344480193, 'penalty': 'l1'}",0.859259,0.870370,0.864198,0.870370,0.871605,0.882716,0.874074,0.843210,0.864198,0.869136,0.866914,0.009935,3
9,0.395561,0.022704,0.003808,0.000368,0.630957,l2,"{'C': 0.630957344480193, 'penalty': 'l2'}",0.849383,0.846914,0.862963,0.870370,0.860494,0.875309,0.859259,0.844444,0.859259,0.865432,0.859383,0.009506,4


In [ ]:
grid_searcher.best_params_

{'C': 10.0, 'penalty': 'l2'}

In [ ]:
lr = grid_searcher.best_estimator_

In [ ]:
lr.fit(vec_x_train, y_train)
pred = lr.predict(vec_x_val)
print(classification_report(pred, y_val))

              precision    recall  f1-score   support

           0       0.94      0.89      0.91      1908
           1       0.76      0.86      0.80       793

    accuracy                           0.88      2701
   macro avg       0.85      0.87      0.86      2701
weighted avg       0.88      0.88      0.88      2701



**Обученный fastText - Без разбики на предложения - Для submission**

In [ ]:
vec_train = df_train['lemmas'].apply(ft_train.get_sentence_vector)

In [ ]:
vec_test = df_test['lemmas'].apply(ft_train.get_sentence_vector)

In [ ]:
vec_df_train = {'vectors': vec_train}
vec_df_train = pd.DataFrame(vec_df_train)
vec_df_train = pd.DataFrame(vec_df_train.vectors.tolist(), index= vec_df_train.index)

In [ ]:
vec_df_test = {'vectors': vec_test}
vec_df_test = pd.DataFrame(vec_df_test)
vec_df_test = pd.DataFrame(vec_df_test.vectors.tolist(), index= vec_df_test.index)

In [ ]:
vec_df_train.shape, vec_df_test.shape

((10809, 300), (3603, 300))

In [ ]:
grid_searcher.fit(vec_df_train, df_train['toxic'])

Fitting 10 folds for each of 12 candidates, totalling 120 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:    1.5s
[Parallel(n_jobs=-1)]: Done  68 tasks      | elapsed:    9.0s
[Parallel(n_jobs=-1)]: Done 120 out of 120 | elapsed:  2.3min finished


GridSearchCV(cv=10, error_score=nan,
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=3000, multi_class='auto',
                                          n_jobs=None, penalty='l2',
                                          random_state=42, solver='liblinear',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='deprecated', n_jobs=-1,
             param_grid={'C': array([1.00000000e-05, 1.58489319e-04, 2.51188643e-03, 3.98107171e-02,
       6.30957344e-01, 1.00000000e+01]),
                         'penalty': ['l1', 'l2']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='accuracy', verbose=5)

In [ ]:
pd.DataFrame(grid_searcher.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_penalty,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
0,0.184797,0.016285,0.004107,0.000875,1e-05,l1,"{'C': 1e-05, 'penalty': 'l1'}",0.666050,0.666667,0.666667,0.666667,0.666667,0.666667,0.665741,0.665741,0.665741,0.665741,0.666235,0.000441,7
1,0.204550,0.013214,0.004048,0.000763,1e-05,l2,"{'C': 1e-05, 'penalty': 'l2'}",0.666050,0.666667,0.666667,0.666667,0.666667,0.666667,0.665741,0.665741,0.665741,0.665741,0.666235,0.000441,7
2,0.174771,0.008126,0.003634,0.000486,0.000158489,l1,"{'C': 0.00015848931924611142, 'penalty': 'l1'}",0.666050,0.666667,0.666667,0.666667,0.666667,0.666667,0.665741,0.665741,0.665741,0.665741,0.666235,0.000441,7
3,0.303806,0.012295,0.005128,0.002095,0.000158489,l2,"{'C': 0.00015848931924611142, 'penalty': 'l2'}",0.666050,0.666667,0.666667,0.666667,0.666667,0.666667,0.665741,0.665741,0.665741,0.665741,0.666235,0.000441,7
4,0.218293,0.014362,0.003881,0.000447,0.00251189,l1,"{'C': 0.0025118864315095794, 'penalty': 'l1'}",0.666050,0.666667,0.666667,0.666667,0.666667,0.666667,0.665741,0.665741,0.665741,0.665741,0.666235,0.000441,7
5,0.290836,0.014232,0.004292,0.000720,0.00251189,l2,"{'C': 0.0025118864315095794, 'penalty': 'l2'}",0.666050,0.666667,0.666667,0.666667,0.666667,0.666667,0.665741,0.665741,0.665741,0.665741,0.666235,0.000441,7
6,0.388411,0.018355,0.004787,0.001436,0.0398107,l1,"{'C': 0.03981071705534969, 'penalty': 'l1'}",0.702128,0.705556,0.704630,0.708333,0.703704,0.701852,0.698148,0.719444,0.705556,0.695370,0.704472,0.006142,6
7,0.405167,0.011827,0.003811,0.000207,0.0398107,l2,"{'C': 0.03981071705534969, 'penalty': 'l2'}",0.740056,0.742593,0.736111,0.746296,0.728704,0.734259,0.728704,0.751852,0.740741,0.728704,0.737802,0.007550,5
8,4.927971,0.118580,0.005571,0.005789,0.630957,l1,"{'C': 0.630957344480193, 'penalty': 'l1'}",0.866790,0.886111,0.862037,0.862963,0.876852,0.876852,0.875000,0.877778,0.868519,0.858333,0.871123,0.008294,3
9,0.529472,0.017843,0.003922,0.000231,0.630957,l2,"{'C': 0.630957344480193, 'penalty': 'l2'}",0.869565,0.877778,0.856481,0.860185,0.870370,0.870370,0.862963,0.870370,0.870370,0.857407,0.866586,0.006572,4


In [ ]:
grid_searcher.best_params_

{'C': 10.0, 'penalty': 'l1'}

In [ ]:
lr = grid_searcher.best_estimator_

In [ ]:
lr.fit(vec_df_train, df_train['toxic'])
pred = lr.predict(vec_df_test)

In [ ]:
submission7 = df_test[['comment_id']]
submission7['toxic'] = pred
submission7.to_csv('submission7.csv', index=False)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
